In [1]:
# TODO NOTEBOOK IMPORTING AND FORMING OF DATA NEEDS TO BE CLEANED; IT'S WAY MESSY

In [2]:
# To be able to use the quantools, due to my crap path names have to add to sys path
import sys
sys.path.insert(0, '/home/adam/Dropbox/2-creations/2-crafts/7-buidl/0-utils/quant_tools/code')

# Import packages
from sklearn.metrics import mutual_info_score
import statsmodels.formula.api as smf
from joblib import Parallel, delayed
from sklearn import decomposition
import pandas_datareader as pdr
import matplotlib.pyplot as plt
from datetime import timedelta
from typing import Dict, List
from tools import QuantTools
from scipy.stats import norm
import statsmodels.api as sm
import scipy.stats as stats
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib
import datetime
import pickle
import random

pd.options.mode.chained_assignment = None

# set color map
viridis = matplotlib.colormaps['viridis']


In [3]:
def importYahoo(ticker: str, start_date: str, end_date: str, rf_df: pd.DataFrame, 
            new_ret_col: str, resample_freq: str='W', rf_col: str='r_rf_tm7') -> pd.DataFrame:
    """ 
    Import Yahoo Finance data for given ticker, time period, taking our risk free rate in rf_df. 
    
    Parameters:
    ticker (str): The ticker symbol to get data for.
    start_date (str): The start date of the data retrieval period.
    end_date (str): The end date of the data retrieval period.
    rf_df (DataFrame): A DataFrame containing risk-free rates.
    new_ret_col (str): The new column name for returns.
    
    Returns:
    DataFrame: A DataFrame containing the date and return data.

    Note: this is done at weekly frequency; would need adjustment for different.
    """
    # import the data
    df = yf.Ticker(ticker).history(period='1d', start=start_date, end=end_date).reset_index()
    
    # reformat
    df['Date'] = pd.to_datetime(df.Date).to_numpy(dtype='datetime64[D]')
    df = df[['Date', 'Close']].rename(columns={'Date': 'date', 'Close': new_ret_col}).set_index('date')
    df = df.resample(resample_freq).last().pct_change().dropna()

    # adjust if resampling monthly
    if resample_freq == 'M':
        df.index = df.index + pd.Timedelta(days=1)

    # take out rf rate
    df = df.merge(rf_df, on='date', how='inner', validate='one_to_one')
    df[new_ret_col] = df[new_ret_col] - df[rf_col]

    # check for NaN values in new_ret_col
    if df[new_ret_col].isnull().values.any():
        print("Warning: NaN values found in return data")
    
    return df[['date', new_ret_col]]


In [4]:
def subsetToAssetUniverse(df: pd.DataFrame, asset_universe_dict: Dict[str, List[str]]) -> pd.DataFrame:
    """
    Subset a DataFrame based on a dictionary of asset universes.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame. Must contain columns "date" and "asset".
    asset_universe_dict : Dict[str, List[str]]
        A dictionary where keys are dates in 'YYYY-MM-DD' format and values are lists of asset names.

    Returns
    -------
    pd.DataFrame
        The subsetted DataFrame.
    """
    # Check that the required columns are present in the DataFrame
    if not set(['date', 'asset']).issubset(df.columns):
        raise ValueError('Input DataFrame must contain "date" and "asset" columns.')

    # Ensure that the 'date' column is of datetime type
    if df['date'].dtype != 'datetime64[ns]':
        df['date'] = pd.to_datetime(df['date'])

    # Loop over all months with their relevant assets
    for key, values in asset_universe_dict.items():
        # Extract the year and month from the key
        year, month = key.split('-')[:2]

        # Drop rows from the dataframe which match the year and month but not the assets
        df = df[~((df.date.dt.year == int(year)) 
                    & (df.date.dt.month == int(month)) 
                    & (~df.asset.isin(values)))]

    return df


In [5]:
def plotReturnHistograms(df: pd.DataFrame, out_fp: str):
    """
    This function takes a DataFrame containing time series data for returns of
    btc, eth, and the cmkt and saves a histogram plot for all three to given fp.
    Each histogram also includes a normal distribution fit.

    Parameters:
    df (pd.DataFrame): A DataFrame containing columns 'date', 'asset', 'char_r_tm7', 'macro_cmkt_tm7'.
    out_fp (str): A string specifying the filepath where the plot should be saved.

    Returns:
    None
    """
    # extract relevant returns
    btc_df  = df[df.asset=='btc'][['date', 'char_r_tm7']]
    btc_df  = btc_df.rename(columns={'char_r_tm7': 'btc'})
    eth_df  = df[df.asset=='eth'][['date', 'char_r_tm7']]
    eth_df  = eth_df.rename(columns={'char_r_tm7': 'eth'})
    cmkt_df = df.groupby('date')[['macro_cmkt_tm7']].mean().reset_index()
    cmkt_df = cmkt_df.rename(columns={'macro_cmkt_tm7': 'cmkt'})

    # form single dataframe
    hist_df = cmkt_df.merge(btc_df, on='date', how='inner', validate='one_to_one')
    hist_df = hist_df.merge(eth_df, on='date', how='inner', validate='one_to_one')

    # initiate the plot with given colors and columns
    fig, axs = plt.subplots(3, sharex=True, sharey=True, figsize=(6.4,4), facecolor='none')
    colors = plt.get_cmap('viridis')(np.linspace(0, 10))
    data_columns = ['cmkt', 'btc', 'eth']

    # plot the data with the normal dist fit
    for idx, ax in enumerate(axs):
        data = hist_df[data_columns[idx]]
        n, bins, patches = ax.hist(data, bins=30, color=colors[idx], alpha=1)

        # Fit a normal distribution
        mu, std = norm.fit(data)

        # Scale normal distribution to histogram
        scale = n.max() / norm.pdf(mu, mu, std).max()
        
        # Plot the PDF
        xmin, xmax = ax.get_xlim()
        x = np.linspace(xmin, xmax, 100)
        p = norm.pdf(x, mu, std) * scale
        ax.plot(x, p, 'k', linewidth=2)

        for spine in ax.spines.values():
            spine.set_visible(False)

    # tighen up the plot
    fig.tight_layout()

    # adjust x axis labels
    plt.xticks(np.arange(-.5, 0.7, 0.1))

    # output
    plt.savefig(out_fp)

    # close the figure
    plt.close(fig)


In [6]:
def plotCumulativeReturns(df: pd.DataFrame, out_fp: str) -> None:
    """
    Plot the time series of cumulative returns to the given output filepath.

    Args:
        df (pd.DataFrame): DataFrame containing the time series data
        out_fp (str): a relative filepath to save the figure to.

    Returns:
        None
    """
    # initialize df with timeserieses to plot
    plot_df = pd.DataFrame(data={'date': []})

    # find all assets present in the panel
    assets = list(np.unique(df.asset.values))

    # form each asset's cumulative return
    for asset in assets:
        # extract asset's returns
        temp_df = df[df.asset==asset][['date', 'char_r_tm7']]

        # ensure it is sorted
        temp_df = temp_df.sort_values(by='date', ignore_index=True)

        # form cumulative return
        temp_df[asset] = (1 + temp_df['char_r_tm7']).cumprod()

        # merge on results
        plot_df = plot_df.merge(temp_df[['date', asset]], on='date', how='outer', validate='one_to_one')

    # form the cmkt return
    temp_df = df[df.asset=='btc'][['date', 'macro_cmkt_tm7']]
    temp_df = temp_df.sort_values(by='date', ignore_index=True)
    temp_df['cmkt'] = (1 + temp_df['macro_cmkt_tm7']).cumprod()
    plot_df = plot_df.merge(temp_df[['date', 'cmkt']], on='date', how='outer', validate='one_to_one')

    # resort
    plot_df = plot_df.sort_values(by='date', ignore_index=True)

    # set index
    plot_df.set_index('date', inplace=True)

    # Plotting the time series
    plt.figure(figsize=(4*1.61, 4), facecolor='none')

    # Form column list
    columns = list(plot_df.columns)
    columns.remove('btc')
    columns.remove('eth')
    columns.remove('cmkt')
    columns = columns + ['eth', 'btc', 'cmkt']

    # Iterate over the columns and plot each time series
    for column in columns:
        if column == 'btc':
            color = '#FDE725FF'
            linewidth = 2
        elif column == 'eth':
            color = '#2D708EFF'
            linewidth = 2
        elif column == 'cmkt':
            color = '#482677FF'
            linewidth = 2
        else:
            color = 'gray'
            linewidth = 0.5
        plt.plot(plot_df.index, plot_df[column], color=color, linewidth=linewidth)

    # Set y-axis to logarithmic scale
    plt.yscale('log')

    # Remove y-axis minor ticks
    plt.gca().yaxis.set_minor_locator(plt.NullLocator())

    # Customize the plot
    plt.grid(visible=True, which='major', axis='y', linewidth=0.5)
    plt.box(False)

    # Add custom labels for important time series
    plt.text(plot_df.index[-45], plot_df['cmkt'].iloc[-52]+5, 'cmkt', color='#482677FF', fontweight='bold', verticalalignment='center', bbox=dict(facecolor='none', edgecolor='none'))
    plt.text(plot_df.index[-1], plot_df['btc'].iloc[-1]-0.1, 'btc', color='#FDE725FF', fontweight='bold', verticalalignment='center', bbox=dict(facecolor='none', edgecolor='none'))
    plt.text(plot_df.index[-1], plot_df['eth'].iloc[-1]+0.3, 'eth', color='#2D708EFF', fontweight='bold', verticalalignment='center', bbox=dict(facecolor='none', edgecolor='none'))

    # output
    plt.savefig(out_fp)
    plt.close()


In [7]:
def genSummaryStatistics(df: pd.DataFrame, lhs_col: str, out_fp: str, periods_in_year: int) -> None:
    """
    Generates summary statistics for the panel and saves them to an Excel file.

    :param df: DataFrame containing asset return data.
    :param lhs_col: Column in df that contains the return data.
    :param out_fp: Output file path for the Excel file.
    :param periods_in_year: integer number of observations in a calendar year.
    """
    # define function for calculating return statistics
    def calcReturnStats(temp_df: pd.DataFrame, asset: str, return_col: str, periods_in_year: int) -> dict:
        mean_return = QuantTools.calcTSAvgReturn(temp_df[return_col].values, annualized=True, periods_in_year=periods_in_year)
        std_dev = QuantTools.calcSD(temp_df[return_col].values, annualized=True, periods_in_year=periods_in_year)
        sharpe_ratio = QuantTools.calcSharpe(temp_df[return_col].values, periods_in_year=periods_in_year)
        skewness = stats.skew(temp_df[return_col].values) / np.sqrt(52)
        kurtosis = stats.kurtosis(temp_df[return_col].values) / 52
        perc_return_above_zero = np.sum(temp_df[return_col]>0) / len(temp_df)
        
        return {'asset': asset,
            'Mean': mean_return,
            'SD': std_dev,
            'Sharpe': sharpe_ratio,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'Pct pos': perc_return_above_zero}

    # drop to only necessary columns
    df = df[['date', 'asset', lhs_col, 'char_size_t', 'char_volume_sum_tm7', 
        'macro_snp500_t', 'macro_dgs1mo_t']].copy()

    # form btc and eth returns
    btc_df = df[df.asset=='btc'].set_index('date')[[lhs_col]]
    eth_df = df[df.asset=='eth'].set_index('date')[[lhs_col]]

    # form cmkt return
    df['weighted_return'] = df[lhs_col] * df['char_size_t']
    total_market_cap = df.groupby('date')['char_size_t'].sum()
    cmkt_df = df.groupby('date')['weighted_return'].sum() / total_market_cap
    cmkt_df = pd.DataFrame(cmkt_df).rename(columns={0: 'return'})

    # import nasdaq data and take out risk free rate
    rf_df = df[['date', 'macro_dgs1mo_t']].drop_duplicates()
    rf_df['r_rf_tm7'] = (1 + rf_df.macro_dgs1mo_t.values / 100) ** (1 / (365 / 7)) - 1
    rf_df = rf_df[['date', 'r_rf_tm7']]
    nsdq_df = importYahoo('^IXIC', '2017-12-29', '2022-12-31', rf_df, 'r_nsdq_tm7')    

    # calc return statistics
    cmkt_stats = calcReturnStats(cmkt_df, 'CMKT', 'return', periods_in_year)
    btc_stats  = calcReturnStats(btc_df, 'Bitcoin', lhs_col, periods_in_year)
    eth_stats  = calcReturnStats(eth_df, 'Ethereum', lhs_col, periods_in_year)
    nsdq_stats = calcReturnStats(nsdq_df, 'Nasdaq', 'r_nsdq_tm7', periods_in_year)
    ret_df = pd.DataFrame([cmkt_stats, btc_stats, eth_stats, nsdq_stats])
    
    # calc extreme event statistics
    ext_data = {'threshold': [], 'count': [], 'percent': []}
    num_obs  = len(cmkt_df)
    for threshold in [-.3, -.2, -.1, -.05, .05, .1, .2, .3]:
        ext_data['threshold'].append(threshold)
        if threshold < 0:
            count = (cmkt_df['return'] < threshold).sum()
            ext_data['count'].append(count)
            ext_data['percent'].append(count / num_obs)
        else:
            count = (cmkt_df['return'] > threshold).sum()
            ext_data['count'].append(count)
            ext_data['percent'].append(count / num_obs)
    ext_df = pd.DataFrame(ext_data)
    
    # calculate yearly stats of unique assets and median mcap and volume
    df['year'] = df['date'].dt.year
    yr_df = pd.DataFrame({
        'num_unique_assets': df.groupby(['year'])['asset'].nunique(),
        'median_market_cap': df.groupby(['year'])['char_size_t'].median(),
        'median_weekly_asset_volume': df.groupby(['year'])['char_volume_sum_tm7'].median()}).reset_index()
    all_df = pd.DataFrame({
        'num_unique_assets': [df['asset'].nunique()],
        'median_market_cap': [df['char_size_t'].median()],
        'median_weekly_asset_volume': [df['char_volume_sum_tm7'].median()]})
    all_df['year'] = 'all'
    yr_df = pd.concat([yr_df, all_df])

    # calculate the total mcap in the last week of each year
    max_dates = df.groupby('year')['date'].max()
    filtered_df = df[df['date'].isin(max_dates)]
    total_mcap_by_year = filtered_df.groupby('year')[['char_size_t']].sum().reset_index()
    yr_df = yr_df.merge(total_mcap_by_year, on='year', how='outer', validate='one_to_one')

    # extract yearly returns
    cmkt_df = cmkt_df.reset_index()
    cmkt_df['year'] = cmkt_df.date.dt.year
    for year in [2018, 2019, 2020, 2021, 2022]:
        yr_df.loc[yr_df.year==year, 'cmkt_ret'] = ((cmkt_df[cmkt_df.year==year]['return']+1).cumprod()-1).values[-1]
    yr_df.loc[yr_df.year=='all', 'cmkt_ret'] = ((cmkt_df['return']+1).cumprod()-1).values[-1]
    
    # save results
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        ret_df.to_excel(writer, sheet_name='raw_ret_stats')
        ext_df.to_excel(writer, sheet_name='raw_extreme_stats')
        yr_df.to_excel(writer, sheet_name='raw_yearly_stats')


In [8]:
def plotRollingSharpe(out_fp: str) -> None:
    """ Plot rolling four year sharpe ratio with new data for the study period. """
    
    # Obtain risk-free rate
    rf_df = pdr.DataReader('DGS1MO', 'fred', '2013-12-31').reset_index()
    rf_df['r_rf_tm7'] = (1 + rf_df.DGS1MO.values / 100) ** (1 / (365 / 7)) - 1
    rf_df['date'] = pd.to_datetime(rf_df.DATE)
    rf_df = rf_df[['date', 'r_rf_tm7']]
    rf_df.set_index('date', inplace=True)
    date_range = pd.date_range(start=rf_df.index.min(), end=rf_df.index.max(), freq='D')
    rf_df = rf_df.reindex(date_range)
    rf_df['r_rf_tm7'].fillna(method='ffill', inplace=True)
    rf_df.reset_index(inplace=True)
    rf_df = rf_df.rename(columns={'index': 'date'})

    # import other asset class data
    start_date = '2013-12-29'
    end_date   = '2022-12-31'
    nsdq_df = importYahoo('^IXIC', start_date, end_date, rf_df, 'Nasdaq')
    snp_df  = importYahoo('^GSPC', start_date, end_date, rf_df, 'SnP 500')
    vt_df   = importYahoo('VT', start_date, end_date, rf_df, 'Global Stocks')
    bnd_df  = importYahoo('BND', start_date, end_date, rf_df, 'US Bonds')
    real_df = importYahoo('VNQ', start_date, end_date, rf_df, 'US Real Estate')
    emrg_df = importYahoo('EBND', start_date, end_date, rf_df, 'Emerging Currencies')
    gld_df  = importYahoo('GLD', start_date, end_date, rf_df, 'Gold')

    # import the btc data and form weekly excess returns
    btc_df = pd.read_csv('../data/raw/XBTUSD_1440.csv',
                        header=None)
    btc_df[0] = pd.to_datetime(btc_df[0], unit='s')
    btc_df = btc_df[[0, 4]]
    btc_df = btc_df.rename(columns={0: 'date', 4: 'price'})
    btc_df.set_index('date', inplace=True)
    btc_df = btc_df.resample('W').last()
    btc_df['Bitcoin'] = btc_df['price'].pct_change()
    btc_df = btc_df.merge(rf_df, on='date', how='inner', validate='one_to_one')
    btc_df['Bitcoin'] = btc_df['Bitcoin'] - btc_df.r_rf_tm7
    btc_df = btc_df.drop(['price', 'r_rf_tm7'], axis=1)

    # form single dataframe
    df = btc_df.merge(nsdq_df, on='date', how='inner', validate='one_to_one')
    for temp_df in [snp_df, vt_df, bnd_df, real_df, emrg_df, gld_df]:
        df = df.merge(temp_df, on='date', how='inner', validate='one_to_one')

    # form sharpe ratios
    window_size = 208
    df.set_index('date', inplace=True)
    columns = list(df.columns.values)
    for col in columns:
        df[col] = np.sqrt(52) * df[col].rolling(window_size).mean() / df[col].rolling(window_size).std()

    # subset to relevant time period
    df = df[(df.index.year >= 2018)
        & (df.index.year <= 2022)]

    # form sharpe ratio plot
    plt.figure(figsize=(4*1.61, 4))
    df.plot(cmap='viridis')
    plt.box(False)
    plt.grid(visible=True, which='major', axis='x', linewidth=0.5)
    #plt.title('Sharpe Ratios: Bitcoin vs Major Asset Classes$^{12}$')
    plt.legend(labels=df.columns.values, 
                loc='lower center',
                ncol=3,
                bbox_to_anchor=(0.5, -0.37),
                frameon=False)
    plt.xlabel("")
    plt.tick_params(axis='both', which='both', bottom=False)
    plt.xticks(rotation=0, ha='center')
    plt.savefig(out_fp, bbox_inches='tight')
    plt.close()


In [9]:
def plotTransactionStats(df: pd.DataFrame, out_fp: str) -> None:
    """ Plot rolling transaction statistics for Bitcoin. """
    # Extract relevant transaction data
    tx_df = df[['date', 'macro_btc_fee_med_usd_t', 'macro_btc_tx_tfr_val_adj_usd_t']].drop_duplicates().copy()

    # reformat the data
    tx_df.set_index('date', inplace=True)
    temp1_df = tx_df[['macro_btc_tx_tfr_val_adj_usd_t']].resample('M').sum()
    temp2_df = tx_df[['macro_btc_fee_med_usd_t']].resample('M').median()
    r_df = temp1_df.merge(temp2_df, how='inner', left_index=True, right_index=True, validate='one_to_one')
    r_df = r_df.rename(columns={'macro_btc_fee_med_usd_t': 'Median Fee (USD)',
                                'macro_btc_tx_tfr_val_adj_usd_t': 'Monthly Volume (USD)'})
    r_df['date'] = r_df.index
    r_df['date'] = r_df.date.dt.strftime('%Y-%m')
    r_df = r_df.set_index('date')

    # plot the data
    plt.figure(figsize=(4*1.61, 4))
    r_df.plot(color=[viridis.colors[0], viridis.colors[111]])
    plt.box(False)
    plt.grid(visible=True, which='major', axis='x', linewidth=0.5)
    plt.legend(labels=r_df.columns.values, 
            loc='lower center',
            ncol=3,
            bbox_to_anchor=(0.5, -0.22),
            frameon=False)
    plt.yscale("log")
    plt.xlabel("")
    plt.xticks(np.array([0,12,24,36,48]), 
            ['2018', '2019', '2020', '2021', '2022'],
            rotation=0, ha='center')
    plt.tick_params(axis='both', which='both', bottom=False)
    plt.savefig(out_fp, bbox_inches='tight')
    plt.close()


In [10]:
def plotHodlingStats(df: pd.DataFrame, out_fp: str) -> None:
    # form the hodl data
    utxo_df = df[['date', 'macro_btc_utxo_age_med_t']].copy()
    utxo_df = utxo_df.drop_duplicates()
    utxo_df = utxo_df.set_index('date')
    utxo_df = utxo_df.rename(columns={'macro_btc_utxo_age_med_t':
                                    'UTXO Median Age (Days)'})

    # Plot the hodl data
    plt.figure(figsize=(4*1.61, 4))
    utxo_df.plot(color=[viridis.colors[0]],
                legend=None)
    plt.box(False)
    plt.grid(visible=True, which='major', axis='x', linewidth=0.5)
    plt.xlabel("")
    plt.tick_params(axis='both', which='both', bottom=False)
    plt.savefig(out_fp, bbox_inches='tight')
    plt.close()


In [11]:
def genForkStatistics(df: pd.DataFrame, out_fp: str) -> None:
    """ Generate event studies for BTC fork dates. """
    # PARAMETERS
    num_bs_samples = int(1e6)
    num_cpus       = 22
    window_size    = 8

    # DEFINE RELEVANT FORKS
    forks = {'bitcoin-21': datetime.datetime(2016,4,17),
            'zcash': datetime.datetime(2016,10,28),
            'bitcoin-cash': datetime.datetime(2017,7,31),
            'bitcoin-gold': datetime.datetime(2017,10,24),
            'bitcoin-diamond': datetime.datetime(2017,11,24),
            'lightning-bitcoin': datetime.datetime(2017,12,18),
            'bitcoinfast': datetime.datetime(2017,12,26),
            'bitcoin2': datetime.datetime(2017,12,28),
            'bitcoin-plus': datetime.datetime(2018,1,2),
            'bitcoin-interest': datetime.datetime(2018,1,22),
            'bitcoin-atom': datetime.datetime(2018,1,24),
            'bitcoin-private': datetime.datetime(2018,2,28),
            'microbitcoin': datetime.datetime(2018,5,29),
            'bitcoin-bep2': datetime.datetime(2018,6,29),
            'bitcoin-sv': datetime.datetime(2018,11,11)}

    # PULL IN OLD DATA TO BUILD RELEVANT TIMESERIES FOR BTC
    df = pd.read_csv('../data/raw/cmc_price_vol_mcap_panel.csv')
    df = df[df.cmc_id==1]
    df = df[['date', 'usd_per_token', 'usd_volume_24h']]
    df['date'] = pd.to_datetime(df.date)
    df = df.drop_duplicates(subset='date')
    df = df.reset_index(drop=True)
    san_df = pd.read_pickle('../data/raw/santiment_panel.pkl')
    san_df = san_df[san_df.san_slug=='bitcoin'][['date', 'active_addresses_24h', 'github_activity',
                                                'social_volume_total']].reset_index(drop=True)
    hash_df = pd.read_excel('../data/raw/coinmetrics_btc_hashrate.xlsx')
    hash_df = hash_df.rename(columns={'Time': 'date',
                                    'BTC / Mean Hash Rate': 'hash_rate'})
    hash_df['date'] = pd.to_datetime(hash_df['date'])
    df = df.merge(san_df,
                on=['date'],
                how='left',
                validate='one_to_one')
    df = df.merge(hash_df,
                on='date',
                how='left',
                validate='one_to_one')
    del san_df, hash_df

    # CLEAN UP THE PANEL

    # ensure it has all days
    sdate   = datetime.date(2015, 1, 1) 
    edate   = datetime.date(2021, 12, 31)  
    delta   = edate - sdate 
    days    = []
    for i in range(delta.days + 1):
        days.append(sdate+timedelta(days=i))
    days_df = pd.DataFrame(data={'date':days})
    df['date'] = df.date.dt.date
    df      = df.merge(days_df,
                    on='date',
                    how='outer',
                    validate='one_to_one')
    df = df.sort_values('date')
    df = df.interpolate()

    # form pct change in all columns and clean up improper values
    cols = list(df.columns.values)
    cols.remove('date')
    for col in cols:
        df[col] = df[col].pct_change()
    df = df.dropna()
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)

    # subset to time period of interest 
    df['date'] = pd.to_datetime(df['date'])
    df[df.date.dt.year <= 2021]
    df = df.reset_index(drop=True)

    # rename columns
    df = df.rename(columns = {'usd_per_token': 'return',
                            'usd_volume_24h': 'usd_volume',
                            'active_addresses_24h': 'active_addresses',
                            'github_activity': 'developer_activity',
                            'social_volume_total': 'social_volume'})

    # INITIALIZE RESULTS 
    cols = list(df.columns.values)
    cols.remove('date')
    results_df = pd.DataFrame(data={'stat': cols,
                                    'window': np.repeat('7 days', len(cols)),
                                    'est': np.zeros(len(cols)),
                                    'se': np.zeros(len(cols))})

    # CALC STAT
    for col in cols:
        diffs = []
        for i in range(len(forks)):
            fork_date = list(forks.values())[i]
            pre_date  = fork_date-pd.Timedelta(window_size, unit="d")
            post_date = fork_date+pd.Timedelta(window_size, unit="d")
            pre_mean  = np.mean(df[(df.date >= pre_date) & (df.date < fork_date)][col])
            post_mean = np.mean(df[(df.date > fork_date) & (df.date <= post_date)][col])
            diff      = post_mean-pre_mean
            diffs.append(diff)
        results_df.loc[results_df.stat==col, 'est'] = np.mean(diffs)

    # CALC STANDARD ERROR (NOTE: 6 MIN RUN TIME)
    num_forks = len(forks)
    rel_dates = list(df.date.values)[window_size:-window_size]
    for col in cols:
        # calc all diffs across the panel
        diffs = []
        for date in rel_dates:
            pre_date  = date-pd.Timedelta(31, unit="d")
            post_date = date+pd.Timedelta(31, unit="d")
            pre_mean  = np.mean(df[(df.date >= pre_date) & (df.date < date)][col])
            post_mean = np.mean(df[(df.date > date) & (df.date <= post_date)][col])
            diff      = post_mean-pre_mean
            diffs.append(diff)

        # reorder the list to randomize
        random.shuffle(diffs)

        # calc bootstrap distribution
        def loopOverNumberBootstrapSamples(i):
            random_diffs = []
            for j in range(num_forks):
                index = np.random.randint(low=0,high=len(diffs))
                diff  = diffs[index]
                random_diffs.append(diff)
            return np.mean(random_diffs)
        bs_stat = Parallel(n_jobs=num_cpus)(delayed(loopOverNumberBootstrapSamples)(i) for i in range(num_bs_samples))

        # calc standard error and add it to results
        se = np.std(bs_stat)
        results_df.loc[results_df.stat==col, 'se'] = se

    # OUTPUT
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        results_df.to_excel(writer, sheet_name='raw_forks')


In [12]:
def genCorrStatistics(df: pd.DataFrame, out_fp: str) -> None:
    """ Build table of correlation statistics. """
    # Set dates
    start_date = '2013-12-01'
    end_date   = '2023-07-01'

    # Obtain risk-free rate
    rf_df = pdr.DataReader('DGS1MO', 'fred', '2013-12-31').reset_index()
    rf_df['r_rf_tm7'] = (1 + rf_df.DGS1MO.values / 100) ** (1 / (365 / 7)) - 1
    rf_df['date'] = pd.to_datetime(rf_df.DATE)
    rf_df = rf_df[['date', 'r_rf_tm7']]
    rf_df.set_index('date', inplace=True)
    date_range = pd.date_range(start=rf_df.index.min(), end=rf_df.index.max(), freq='D')
    rf_df = rf_df.reindex(date_range)
    rf_df['r_rf_tm7'].fillna(method='ffill', inplace=True)
    rf_df.reset_index(inplace=True)

    rf_df = rf_df.rename(columns={'index': 'date'})

    # Import other asset class data
    nsdq_df = importYahoo('^IXIC', start_date, end_date, rf_df, 'Nasdaq')
    snp_df  = importYahoo('^GSPC', start_date, end_date, rf_df, 'SnP 500')
    rus_df  = importYahoo('^RUT', start_date, end_date, rf_df, 'Russell 2000')
    vt_df   = importYahoo('VT', start_date, end_date, rf_df, 'Global Stocks')
    bnd_df  = importYahoo('BND', start_date, end_date, rf_df, 'US Bonds')
    bndx_df = importYahoo('BNDX', start_date, end_date, rf_df, 'Ex-US Global Bonds')
    real_df = importYahoo('VNQ', start_date, end_date, rf_df, 'US Real Estate')
    emrg_df = importYahoo('EBND', start_date, end_date, rf_df, 'Emerging Currencies')
    dbc_df  = importYahoo('DBC', start_date, end_date, rf_df, 'Commodities')
    gld_df  = importYahoo('GLD', start_date, end_date, rf_df, 'Gold')

    # Form crypto time series
    btc_df = df[df.asset=='btc'][['date', 'char_r_tm7']].reset_index(drop=True).copy()
    btc_df = btc_df.rename(columns={'char_r_tm7': 'Bitcoin'})

    eth_df = df[df.asset=='eth'][['date', 'char_r_tm7']].reset_index(drop=True).copy()
    eth_df = eth_df.rename(columns={'char_r_tm7': 'Ethereum'})

    cmkt_df = df.groupby('date')[['macro_cmkt_tm7']].mean().reset_index()
    cmkt_df = cmkt_df.rename(columns={'macro_cmkt_tm7': 'Crypto Market'})


    # form single dataframe
    t_df = cmkt_df.merge(btc_df, on='date', how='inner', validate='one_to_one')
    for temp_df in [eth_df, nsdq_df, snp_df, rus_df, vt_df, bnd_df, bndx_df, real_df, emrg_df, dbc_df, gld_df]:
        t_df = t_df.merge(temp_df, on='date', how='inner', validate='one_to_one')
    t_df = t_df.set_index('date')

    # Export
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        t_df[(t_df.index >= '2018-01-01') 
            & (t_df.index <= '2022-12-31')].corr().to_excel(writer, sheet_name='raw_corr')


In [13]:
def plotRollingCorrelations(out_fp: str) -> None:
    """ Plot figure out rolling correlations between BTC and other asset classes. """
    # Parameters
    start_date = '2013-11-01'
    end_date   = '2023-07-02'
    rolling_window = 48

    # Pull and form exp inf data
    ei_df = pdr.DataReader('EXPINF1YR', 'fred', start_date).reset_index()
    ei_df = ei_df.rename(columns={'DATE': 'date'})
    ei_df = ei_df.set_index('date')
    ei_df = ei_df.pct_change().dropna().reset_index()
    ei_df = ei_df[(ei_df.date >= '2014-01-01') & (ei_df.date <= '2023-07-01')]

    # Obtain risk-free rate
    rf_df = pdr.DataReader('DGS1MO', 'fred', '2013-12-15').reset_index()
    rf_df['r_rf'] = (1 + rf_df.DGS1MO.values / 100) ** (1 / 12) - 1
    rf_df['date'] = pd.to_datetime(rf_df.DATE)
    rf_df = rf_df[['date', 'r_rf']]
    rf_df.set_index('date', inplace=True)
    date_range = pd.date_range(start=rf_df.index.min(), end=rf_df.index.max(), freq='D')
    rf_df = rf_df.reindex(date_range)
    rf_df['r_rf'].fillna(method='ffill', inplace=True)
    rf_df.reset_index(inplace=True)
    rf_df = rf_df.rename(columns={'index': 'date'})
    rf_df = rf_df[(rf_df.date >= '2014-01-01') & (rf_df.date <= '2023-07-01')]

    # Import other asset class data
    nsdq_df = importYahoo('^IXIC', start_date, end_date, rf_df, 'Nasdaq', 'M', 'r_rf')
    bnd_df  = importYahoo('BND', start_date, end_date, rf_df, 'US Bonds', 'M', 'r_rf')
    real_df = importYahoo('VNQ', start_date, end_date, rf_df, 'US Real Estate', 'M', 'r_rf')
    emrg_df = importYahoo('EBND', start_date, end_date, rf_df, 'Emerging Currencies', 'M', 'r_rf')
    dbc_df  = importYahoo('DBC', start_date, end_date, rf_df, 'Commodities', 'M', 'r_rf')
    gld_df  = importYahoo('GLD', start_date, end_date, rf_df, 'Gold', 'M', 'r_rf')

    # import the btc data and form monthly excess returns
    btc_df = pd.read_csv('../data/raw/cm_btc.csv')
    btc_df['date'] = pd.to_datetime(btc_df.date)
    btc_df = btc_df.rename(columns={'btc': 'Bitcoin'})
    btc_df.set_index('date', inplace=True)
    btc_df = btc_df.resample('M').last()
    btc_df.index = btc_df.index + pd.Timedelta(days=1)
    btc_df['Bitcoin'] = btc_df['Bitcoin'].pct_change()
    btc_df = btc_df.merge(rf_df, on='date', how='inner', validate='one_to_one')
    btc_df['Bitcoin'] = btc_df['Bitcoin'] - btc_df.r_rf
    btc_df = btc_df.drop(['r_rf'], axis=1)

    # form single dataframe
    df = btc_df.merge(nsdq_df, on='date', how='inner', validate='one_to_one')
    for temp_df in [bnd_df, real_df, emrg_df, dbc_df, gld_df, ei_df]:
        df = df.merge(temp_df, on='date', how='inner', validate='one_to_one')
    df = df.set_index('date')

    # Calc rolling correlations
    results_df = pd.DataFrame(data = {'date': df.index.values})
    results_df = results_df.set_index('date')
    col_list = list(df.columns.values)[1:]
    for col in col_list:
        temp_df = df[['Bitcoin']].rolling(rolling_window).corr(df[col])
        temp_df = temp_df.rename(columns = {'Bitcoin': col})
        temp_df = temp_df.dropna()
        results_df = results_df.merge(temp_df,
                                    how='inner',
                                    left_index=True,
                                    right_index=True,
                                    validate='one_to_one')
    results_df = results_df[results_df.index < '2023-01-01']

    # Form figure
    plt.figure(figsize=(4*1.61, 4))
    results_df.plot(color=[viridis.colors[0], viridis.colors[44], viridis.colors[88],
        viridis.colors[132], viridis.colors[178], viridis.colors[222], viridis.colors[255]])
    plt.box(False)
    plt.grid(visible=True, which='major', axis='x', linewidth=0.5)
    plt.legend(labels=results_df.columns.values, 
                loc='lower center',
                ncol=3,
                bbox_to_anchor=(0.5, -0.37),
                frameon=False)
    plt.xlabel("")
    plt.tick_params(axis='both', which='both', bottom=False)
    plt.xticks(rotation=0, ha='center')
    plt.savefig(out_fp, bbox_inches='tight')
    plt.close()


In [14]:
def formRiskReturnScatterPlot(df: pd.DataFrame, out_fp: str, periods_in_year: int) -> None:
    """ Plot all assets in return risk space. """
    # calc each asset's annualized return and risk
    r_df = df[['date', 'asset', 'char_r_tm7', 'macro_dgs1mo_t']].copy()
    r_df['char_r_tm7'] = r_df.char_r_tm7 + ((r_df.macro_dgs1mo_t.values/100)+1)**(1/52)-1
    r_df = r_df.drop('macro_dgs1mo_t', axis=1)
    r_df['annual_return'] = r_df.groupby('asset')['char_r_tm7'].transform(lambda x: 
                                QuantTools.calcCumulativeReturn(x, annualized=True, periods_in_year=periods_in_year))
    r_df['annual_risk'] = r_df.groupby('asset')['char_r_tm7'].transform(lambda x: 
                                QuantTools.calcSD(x, annualized=True, periods_in_year=periods_in_year))
    r_df = r_df[['asset', 'annual_return', 'annual_risk']].drop_duplicates()
    r_df = r_df.sort_values(by='asset', ignore_index=True)

    # drop assets with absurd returns and note what/how many we dropped
    crazy_df = r_df[r_df.annual_return > 5].copy()
    r_df = r_df[r_df.annual_return <= 5]
    for asset in list(crazy_df.asset.values):
        print(f'{asset} was dropped from risk return plot with annual return {np.round(crazy_df[crazy_df.asset==asset].annual_return.values[0], 1)}')

    # add 1 month tbill to data
    t_df = df[['date', 'macro_dgs1mo_t']].drop_duplicates().reset_index(drop=True)
    t_weekly_returns = ((t_df.macro_dgs1mo_t.values/100)+1)**(1/52)-1
    t_annual_return = QuantTools.calcCumulativeReturn(t_weekly_returns, annualized=True, periods_in_year=periods_in_year)
    r_df = pd.concat([r_df, 
                    pd.DataFrame(data={'asset': ['1moTbill'], 
                                    'annual_return': [t_annual_return],
                                    'annual_risk': [0]})]).reset_index(drop=True)

    # Calc risk free rate
    rf_df = pdr.DataReader('DGS1MO', 'fred', '2013-12-31').reset_index()
    rf_df['r_rf_tm7'] = (1 + rf_df.DGS1MO.values / 100) ** (1 / (365 / 7)) - 1
    rf_df['date'] = pd.to_datetime(rf_df.DATE)
    rf_df = rf_df[['date', 'r_rf_tm7']]
    rf_df.set_index('date', inplace=True)
    date_range = pd.date_range(start=rf_df.index.min(), end=rf_df.index.max(), freq='D')
    rf_df = rf_df.reindex(date_range)
    rf_df['r_rf_tm7'].fillna(method='ffill', inplace=True)
    rf_df.reset_index(inplace=True)
    rf_df = rf_df.rename(columns={'index': 'date'})

    # Import other asset class data
    nsdq_df = importYahoo('^IXIC', '2017-11-30', '2023-01-08', rf_df, 'Nasdaq')
    nsdq_df = nsdq_df[(nsdq_df.date >= '2018-01-01') & (nsdq_df.date <= '2022-12-31')]

    # Add nsdq
    nsdq_annual_ret = QuantTools.calcCumulativeReturn(nsdq_df.Nasdaq.values, annualized=True, periods_in_year=periods_in_year)
    nsdq_sd = QuantTools.calcSD(nsdq_df.Nasdaq.values, annualized=True, periods_in_year=periods_in_year)
    r_df = pd.concat([r_df, 
                    pd.DataFrame(data={'asset': ['nsdq'], 
                            'annual_return': [nsdq_annual_ret],
                            'annual_risk': [nsdq_sd]})]).reset_index(drop=True)

    # Add cmkt
    cmkt_df = df.groupby('date')[['macro_cmkt_tm7']].mean().reset_index()
    cmkt_df = cmkt_df.rename(columns={'macro_cmkt_tm7': 'cmkt'})
    cmkt_df['cmkt'] = cmkt_df.cmkt+t_weekly_returns
    cmkt_annual_ret = QuantTools.calcCumulativeReturn(cmkt_df.cmkt.values, annualized=True, periods_in_year=periods_in_year)
    cmkt_sd = QuantTools.calcSD(cmkt_df.cmkt.values, annualized=True, periods_in_year=periods_in_year)
    r_df = pd.concat([r_df,
                    pd.DataFrame(data={'asset': ['cmkt'], 
                            'annual_return': [cmkt_annual_ret],
                            'annual_risk': [cmkt_sd]})]).reset_index(drop=True)

    # Form optimal risk portfolio
    nsdq_pcts = list(np.arange(0, 1, 0.05)) + [.99]
    max_sharpe = 0
    nsdq_opt   = 0
    for nsdq_pct in nsdq_pcts:
        cmkt_pct = 1 - nsdq_pct
        d_ret = nsdq_df.Nasdaq.values*nsdq_pct + cmkt_df.cmkt.values*cmkt_pct
        d_annual_ret = QuantTools.calcCumulativeReturn(d_ret, annualized=True, periods_in_year=periods_in_year)
        d_sd = QuantTools.calcSD(d_ret, annualized=True, periods_in_year=periods_in_year)
        print(f"for nsdq pct {np.round(nsdq_pct, 2)}, the sharpe is {np.round(d_annual_ret / d_sd, 3)} and annual return is {np.round(d_annual_ret, 4)}")
        if (d_annual_ret / d_sd) > max_sharpe:
            nsdq_opt = nsdq_pct
            opt_return = d_annual_ret
            opt_sd     = d_sd
            max_sharpe = opt_return / opt_sd
        
    print(f"\n for a sharpe of {np.round(max_sharpe, 4)}, allocate {np.round(nsdq_opt, 4)} to nsdq and {np.round(1-nsdq_opt, 4)} to cmkt.")

    opt_col_name = 'nsdq_'+str(np.round(nsdq_opt, 2))+'_cmkt_'+str(np.round(1-nsdq_opt, 2))
    r_df = pd.concat([r_df,
                    pd.DataFrame(data={'asset': [opt_col_name], 
                            'annual_return': [opt_return],
                            'annual_risk': [opt_sd]})]).reset_index(drop=True)

    # Final edits
    r_df = r_df.set_index('asset')
    r_df = r_df[r_df.index != 'luna']

    # Form figure
    plt.figure(figsize=(4*1.61, 4))
    plt.scatter(r_df['annual_risk'], r_df['annual_return'], color=viridis.colors[111])

    labels = ["1moTbill", "btc", "cmkt", "eth", "nsdq", opt_col_name]
    colors = ['lightgrey', viridis.colors[255], viridis.colors[33], 
        viridis.colors[177], 'grey', 'black']
    for label, color in zip(labels, colors):
        asset_df = r_df[r_df.index==label]
        plt.scatter(asset_df['annual_risk'], asset_df['annual_return'], color=color)

    point_rf = r_df[r_df.index == "1moTbill"].iloc[0]
    point_nsdq = r_df[r_df.index == "nsdq"].iloc[0]
    point_cmkt = r_df[r_df.index == "cmkt"].iloc[0]
    point_opt  = r_df[r_df.index == opt_col_name].iloc[0]
    slope_nsdq = (point_nsdq['annual_return'] - point_rf['annual_return']) / (point_nsdq['annual_risk'] - point_rf['annual_risk'])
    slope_cmkt = (point_cmkt['annual_return'] - point_rf['annual_return']) / (point_cmkt['annual_risk'] - point_rf['annual_risk'])
    slope_opt = (point_opt['annual_return'] - point_rf['annual_return']) / (point_opt['annual_risk'] - point_rf['annual_risk'])
    max_x = r_df['annual_risk'].max()
    max_y_nsdq = slope_nsdq * (max_x - point_rf['annual_risk']) + point_rf['annual_return']
    max_y_cmkt = slope_cmkt * (max_x - point_rf['annual_risk']) + point_rf['annual_return']
    max_y_opt  = slope_opt  * (max_x - point_rf['annual_risk']) + point_rf['annual_return']
    plt.plot([point_rf['annual_risk'], max_x], [point_rf['annual_return'], max_y_nsdq], ':', color='grey')
    plt.plot([point_rf['annual_risk'], max_x], [point_rf['annual_return'], max_y_cmkt], ':', color=viridis.colors[33])
    plt.plot([point_rf['annual_risk'], max_x], [point_rf['annual_return'], max_y_opt], ':', color='black')

    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.xlim(0, max_x + 0.3)

    plt.savefig(out_fp, bbox_inches='tight')
    plt.close()


In [15]:
def reportICOStatistics(df: pd.DataFrame) -> None:
    # Subset to relevant data
    ico_df = df[['date', 'asset', 'r_ex_tp7', 'char_price_t', 
        'char_ico', 'char_ico_days_since_t', 'char_ico_price']].copy()

    # Determine which assets are ico assets
    ico_assets = np.unique(ico_df[ico_df.char_ico==1].asset.values)

    # Subset to relevant assets
    ico_df = ico_df[ico_df.asset.isin(ico_assets)]
    ico_df = ico_df.drop(columns='char_ico', axis=1)

    # Form dataframe of the last tradable data for all assets
    last_df = ico_df.groupby('asset')[['date']].max().reset_index()
    last_df = last_df.merge(ico_df[['date', 'asset', 'char_price_t', 'char_ico_price']], 
        on=['date', 'asset'], how='left', validate='one_to_one')

    # Calc and report return since ICO
    last_df['r_from_ico'] = last_df.char_price_t / last_df.char_ico_price - 1
    return_from_ico_dates = last_df.r_from_ico.mean()
    print(f"Return on ICO assets from ICO date to end of panel is: {int(return_from_ico_dates)}x.")

    # Calc return of ico assets from first tradable data to end of panel
    min_df = ico_df[(ico_df.char_ico_days_since_t>0)].groupby('asset')[['date']].min().reset_index()
    min_df = min_df.merge(ico_df[['date', 'asset', 'char_price_t']],
        on=['date', 'asset'], how='left', validate='one_to_one')
    min_df = min_df.rename(columns={'char_price_t': 'first_trade_price'})
    min_df = min_df.drop('date', axis=1)

    max_df = ico_df.groupby('asset')[['date']].max().reset_index()
    max_df = max_df.merge(ico_df[['date', 'asset', 'char_price_t']], 
        on=['date', 'asset'], how='left', validate='one_to_one')
    max_df = max_df.rename(columns={'char_price_t': 'last_trade_price'})
    max_df = max_df.drop('date', axis=1)
    trade_df = max_df.merge(min_df, on='asset', how='inner', validate='one_to_one')
    trade_df['r'] = trade_df.last_trade_price / trade_df.first_trade_price - 1

    return_from_first_tradable = trade_df.r.mean()
    print(f"Return on ICO assets from first tradable date to end of panel is: {np.round(100*return_from_first_tradable, 1)}%.")


In [16]:
def inflationStatistics(df: pd.DataFrame, out_fp: str) -> None:
    # Copy the raw data for use later as this func is terribly written
    temp_df = df.copy()

    # Params
    start_date = '2013-11-01'
    end_date   = '2023-07-02'

    # Obtain monthly excess returns for cmkt
    cmkt_df = df[['date', 'macro_cmkt_tm7']].drop_duplicates().copy()
    cmkt_df.set_index('date', inplace=True)
    cmkt_df['cmkt'] = cmkt_df.macro_cmkt_tm7+1
    cmkt_df = cmkt_df.drop('macro_cmkt_tm7', axis=1)
    cmkt_df = cmkt_df.resample('M')[['cmkt']].prod()-1
    cmkt_df.index = cmkt_df.index + pd.DateOffset(days=1) - pd.DateOffset(months=1)

    # Pull and form exp inf data
    inf_col = 'EXPINF10YR'
    ei_df = pdr.DataReader(inf_col, 'fred', start_date).reset_index()
    ei_df = ei_df.rename(columns={'DATE': 'date'})
    ei_df = ei_df.set_index('date')
    ei_df[inf_col] = (1+ei_df[inf_col])**(1/12)-1 # annual inflation rate to monthly rate
    ei_df = ei_df.pct_change().dropna().reset_index()
    ei_df = ei_df[(ei_df.date >= '2014-01-01') & (ei_df.date <= '2023-07-01')]

    # Obtain risk-free rate
    rf_df = pdr.DataReader('DGS1MO', 'fred', '2013-12-15').reset_index()
    rf_df['r_rf'] = (1 + rf_df.DGS1MO.values / 100) ** (1 / 12) - 1
    rf_df['date'] = pd.to_datetime(rf_df.DATE)
    rf_df = rf_df[['date', 'r_rf']]
    rf_df.set_index('date', inplace=True)
    date_range = pd.date_range(start=rf_df.index.min(), end=rf_df.index.max(), freq='D')
    rf_df = rf_df.reindex(date_range)
    rf_df['r_rf'].fillna(method='ffill', inplace=True)
    rf_df.reset_index(inplace=True)
    rf_df = rf_df.rename(columns={'index': 'date'})
    rf_df = rf_df[(rf_df.date >= '2014-01-01') & (rf_df.date <= '2023-07-01')]

    # Import other asset class data
    gld_df  = importYahoo('GLD', start_date, end_date, rf_df, 'Gold', 'M', 'r_rf')
    nsdq_df  = importYahoo('^IXIC', start_date, end_date, rf_df, 'Nasdaq', 'M', 'r_rf')

    # import the btc data and form monthly excess returns
    btc_df = pd.read_csv('../data/raw/cm_btc.csv')
    btc_df['date'] = pd.to_datetime(btc_df.date)
    btc_df = btc_df.rename(columns={'btc': 'Bitcoin'})
    btc_df.set_index('date', inplace=True)
    btc_df = btc_df.resample('M').last()
    btc_df.index = btc_df.index + pd.Timedelta(days=1)
    btc_df['Bitcoin'] = btc_df['Bitcoin'].pct_change()
    btc_df = btc_df.merge(rf_df, on='date', how='inner', validate='one_to_one')
    btc_df['Bitcoin'] = btc_df['Bitcoin'] - btc_df.r_rf
    btc_df = btc_df.drop(['r_rf'], axis=1)

    # form single dataframe
    df = cmkt_df.merge(btc_df, on='date', how='right', validate='one_to_one')
    df = df.merge(ei_df, on='date', how='outer', validate='one_to_one')
    df = df.merge(gld_df, on='date', how='outer', validate='one_to_one')
    df = df.merge(nsdq_df, on='date', how='outer', validate='one_to_one')
    df = df.set_index('date')
    df = df[df.index <= '2023-01-01']

    # OVERALL CORR OF MONTHLY RETURNS AND INF INNOV
    print(df.index.min())
    print(df.index.max())
    print(df.drop('cmkt', axis=1).corr())

    # OVERALL CORR OF MONTHLY RETURNS AND INF INNOV JUST 2018-2022
    print(df[df.cmkt.notnull()].corr())

    # CORR BETWEEN BTC AND EXPINF1YR AS WELL AS BTWN GOLD AND EXPINF1YR 
    # ON TOP 12 MONTHS IN 2014 Jan to Dec 2022 and Jan 2018 to Dec 2022 for cmkt
    df['inf_abs'] = np.abs(df[inf_col])
    print(df.drop('cmkt', axis=1).sort_values(by='inf_abs', ascending=False)[:12].corr())
    print(df[df.cmkt.notnull()].sort_values(by='inf_abs', ascending=False)[:12].corr())
    df = df.drop('inf_abs', axis=1)

    # Regress BTC returns on CMKT and inf
    df = df[df.cmkt.notnull()]
    y = df.Bitcoin
    X = df[['cmkt', inf_col]]
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    results_summary = results.summary2().tables
    summary_df = pd.DataFrame(results_summary[1])
    summary_df['N'] = len(y)
    summary_df['R2'] = results.rsquared
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        summary_df.to_excel(writer, sheet_name='raw_inf_reg')

    # Fama Macbeth
    temp_df = temp_df[['date', 'asset', 'char_r_tm7']].copy()
    asset_df = pd.DataFrame()
    assets = list(np.unique(temp_df.asset.values))
    for asset in assets:
        t_df = temp_df[temp_df.asset==asset][['date', 'char_r_tm7']]
        t_df.set_index('date', inplace=True)
        t_df['r'] = t_df['char_r_tm7']+1
        t_df = t_df.drop('char_r_tm7', axis=1)
        t_df = t_df.resample('M')[['r']].prod()-1
        t_df.index = t_df.index + pd.DateOffset(days=1) - pd.DateOffset(months=1)
        t_df = t_df.reset_index()
        t_df['asset'] = asset
        asset_df = pd.concat([asset_df, t_df])

    beta_hats = []
    assets = list(np.unique(asset_df.asset.values))
    assets.remove('inv')
    for asset in assets:
        a_df = asset_df[asset_df.asset==asset]
        a_df = a_df.merge(df[[inf_col]].reset_index(), on='date', how='inner', validate='one_to_one')
        y = a_df.r
        X = a_df[[inf_col]]
        X = sm.add_constant(X)
        model = sm.OLS(y, X)
        results = model.fit()
        beta_hats.append(results.params[1])

    y = asset_df[asset_df.asset!='inv'].groupby('asset')['r'].mean().values
    x = np.array(beta_hats)
    X = sm.add_constant(x)
    model = sm.OLS(y, X)
    results = model.fit()

    results_summary = results.summary2().tables
    summary_df = pd.DataFrame(results_summary[1])
    summary_df['N'] = len(y)
    summary_df['R2'] = results.rsquared
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        summary_df.to_excel(writer, sheet_name='raw_fama_macbeth')


In [17]:
def genCharDescStatistics(df: pd.DataFrame, char_group_cols: List[list], lhs_col: str, out_fp: str) -> None:
    char_cols = [col for sublist in char_group_cols for col in sublist]
    char_df = df[[lhs_col]+char_cols].copy()

    char_desc_stat_df = char_df.describe().T

    percentiles = [1, 5, 95, 99]
    for prcnt in percentiles:
        char_desc_stat_df[str(prcnt)+'%'] = char_df.apply(lambda x: np.percentile(x, prcnt))

    def formatter(x):
        if x > 1000 or x < -1000:
            return "{:.1e}".format(x)
        elif 10 <= x <= 1000 or -1000 <= x <= -10:
            return "{:.0f}".format(x)
        elif 1 <= x < 10 or -10 < x <= -1:
            return "{:.1f}".format(x)
        elif 0.001 <= x < 1 or -1 < x <= -0.001:
            return "{:.3f}".format(x)
        else: # -0.001 < x < 0.001
            return "{:.1e}".format(x)

    for col in list(char_desc_stat_df.columns.values):
        char_desc_stat_df[col] = char_desc_stat_df[col].apply(formatter)
    
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        char_desc_stat_df.to_excel(writer, sheet_name='raw_char_desc_stat')

def generatePrincipalComponent(
    df: pd.DataFrame, var: List[str], ncomponents: int = 1, pc_col_name: str = 'pca') -> pd.DataFrame:
    
    '''
    Generate a column of the principle component among var and add the column to the dataframe
    
    Keyword arguments:
        df -- input dataframe
        var -- list of variables to inspect
        ncomponents -- number of components of PCA
        pc_col_name -- name of the PC column
        
    Return value:
        df -- dataframe with the PC column
    '''
    
    # pca
    X = df[var]
    pca = decomposition.PCA(n_components=ncomponents)
    res = pca.fit_transform(X)

    # add pc to dataframe
    df[pc_col_name] = res
    
    return df

def genPcCorrTable(df: pd.DataFrame, char_group_cols: List[list], out_fp: str) -> None:
    # Form the first PC for each group of characteristics
    pcs = []
    for char_cols in char_group_cols:
        char_df = df[char_cols]
        pca = decomposition.PCA(n_components=1)
        pc  = pca.fit_transform(char_df)
        pcs.append(pc)

    # Concatenate the PC's together
    r_df = pd.DataFrame(np.concatenate(pcs, axis=1))

    # Calc correlations
    corr_df = r_df.corr()

    # Output the results
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        corr_df.to_excel(writer, sheet_name='raw_pca_corr')

def formRelCharDf(df: pd.DataFrame, lhs_col: str, char_cols: List[str], char_name: str) -> pd.DataFrame:
    """
    This function forms a DataFrame with relevant data, rearranges columns, drops data beyond a certain date, 
    and generates a Principal Component column.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    lhs_col (str): The column name to be used as the left-hand side column.
    char_cols (List[str]): A list of character column names.
    char_name (str): The name to be used for the generated Principal Component column.

    Returns:
    pd.DataFrame: The resulting DataFrame after all transformations.
    """
    
    # Form the relevant data
    # -accounting for whether return tm7 is in char_cols to avoiding dup col
    if 'char_r_tm7' in char_cols:
        char_df = df[['date', 'asset', lhs_col]+char_cols].copy()
        char_df['r_ex_tp0'] = char_df['char_r_tm7']
    else:
        char_df = df[['date', 'asset', lhs_col]+['char_r_tm7']+char_cols].copy()
        char_df = char_df.rename(columns={'char_r_tm7': 'r_ex_tp0'})

    # Form remaining LHS columns
    new_lhs_cols = ['r_ex_tp0']
    for horizon, day_delta in zip([14, 30, 60], [14, 28, 56]):
        rhs_col = 'char_r_tm'+str(horizon)
        new_lhs_col = 'r_ex_tp'+str(horizon)
        new_lhs_cols.append(new_lhs_col)
        temp_df = df[['date', 'asset', rhs_col]].copy()
        temp_df = temp_df.rename(columns={rhs_col: new_lhs_col})
        temp_df['date'] = temp_df.date - pd.DateOffset(days=day_delta)
        char_df = char_df.merge(temp_df, 
            on=['date', 'asset'], how='left', validate='one_to_one') 

    # Rearrange cols
    char_df = char_df[['date', 'asset', lhs_col] + new_lhs_cols + char_cols]

    # Drop 2h 2022 data
    char_df = char_df[char_df.date < '2022-07-01']

    # Generate the PC column
    char_df = generatePrincipalComponent(char_df, char_cols, 1, 'pc_'+char_name)

    return char_df
    
def genAndOutputCharCorrTable(char_df: pd.DataFrame,
    char_cols: List[str], char_name: str, out_fp: str) -> None:
    """
    Generate a correlation DataFrame and save it to an Excel file.

    Parameters:
    char_df: The DataFrame to calculate correlations on.
    char_cols: The columns to include in the correlation calculation.
    char_name: The name of the characteristic to include in the correlation calculation.
    out_fp: The file path to save the correlation DataFrame to.
    """
    # Generate correlation dataframe
    corr_df = char_df[char_cols+['pc_'+char_name]].corr()

    # Save the table
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        corr_df.to_excel(writer, sheet_name='raw_char_corr_'+char_name)

def format_float(num):
    str_num = str(num)
    if "e" in str_num or num < 0.01:
        return np.format_float_scientific(num, precision=1, unique=False, trim='k')
    else:
        decimal_part = str_num.split('.')[1] if '.' in str_num else ''
        precision = 2 if len(decimal_part) >= 2 else len(decimal_part)
        return np.format_float_positional(num, precision=precision, unique=False, trim='k')

def genAndOutputCharRegTable(char_df: pd.DataFrame,
    char_cols: List[str], char_name: str, out_fp: str) -> None:
    """
    Generate a regression DataFrame and save it to an Excel file.

    Parameters:
    char_df: The DataFrame to calculate regressions on.
    char_cols: The columns to include in the regression calculation.
    char_name: The name of the characteristic to include in the regression calculation.
    out_fp: The file path to save the regression DataFrame to.
    """
    
    # Initialize DataFrame for the results
    results_df = pd.DataFrame(data={'char': []})

    # Generate results for all horizons and all characteristics
    for horizon in [0, 7, 14, 30, 60]:
        horizon_results_df = pd.DataFrame()
        for char_col in char_cols+['pc_'+char_name]:
            # calc the uni reg
            y_col   = 'r_ex_tp'+str(horizon)
            rel_df  = char_df[char_df[y_col].notnull()]
            y       = rel_df[y_col]
            x       = rel_df[char_col]
            X       = sm.add_constant(x)
            model   = sm.OLS(y, X)
            results = model.fit()

            # obtain the results
            coef    = results.params[1]
            se      = results.bse[1]
            tstat   = results.tvalues[1]
            r2      = results.rsquared

            # format the results
            coef    = format_float(coef)
            se      = "("+format_float(se)+")"
            r2      = np.round(results.rsquared, 2)

            # add significant
            if np.abs(tstat) >= 2.326:
                coef = coef+"***"
            elif np.abs(tstat) >= 1.96:
                coef = coef+"**"
            elif np.abs(tstat) >= 1.645:
                coef = coef+"*"

            # build df of results
            horizon_results_df = pd.concat([horizon_results_df,
                pd.DataFrame(data={
                    'char': [char_col+'_coef', 
                            char_col+'_se', 
                            char_col+'_r2'],
                    'tp'+str(horizon): [coef, se, r2]})])
        
        # build the main df
        results_df = results_df.merge(horizon_results_df, 
            on='char', how='outer', validate='one_to_one')

    # Save the table
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        results_df.to_excel(writer, sheet_name='raw_char_reg_'+char_name)

def calc_MI(temp_df: pd.DataFrame, lhs_col: str, feat_col: str):
    ''' Calculate mutual information between two given columns in given df.
    
    Args:
        temp_df (pd.DataFrame): dataframe to obtain x and y columns from.
        lhs_col (str): name of first column to use to calc MI.
        feat_col (str): name of second col to use to calc MI.

    Returns:
        mi (float): mutual information between two given vectors.
    '''
    # obtain the two data series
    x    = temp_df[lhs_col].values
    y    = temp_df[feat_col].values

    # calc mutual information
    bins = int(np.floor(np.sqrt(temp_df.shape[0]/5)))
    c_xy = np.histogram2d(x, y, bins)[0]
    mi   = mutual_info_score(None, None, contingency=c_xy)

    return mi

def calcMiByYear(df: pd.DataFrame, lhs_col: str, char_group_cols: List[list], out_fp: str) -> None:
    # obtain all characteristics in same order
    char_cols = [col for sublist in char_group_cols for col in sublist]
        
    # initialize object for results
    results_df = pd.DataFrame(data={
        'char': char_cols+['N'],
        '2018': ['' for _ in char_cols]+[''],
        '2019': ['' for _ in char_cols]+[''],
        '2020': ['' for _ in char_cols]+[''],
        '2021': ['' for _ in char_cols]+[''],
        '2022': ['' for _ in char_cols]+[''],
        'All': ['' for _ in char_cols]+['']
    })

    # loop over years to calc mutual info
    for col in char_cols:
        for year in [2018, 2019, 2020, 2021, 2022]:
            mi = calc_MI(df[(df.date.dt.year==year)], lhs_col, col)
            results_df.loc[results_df.char==col, str(year)] = mi
        mi = calc_MI(df, lhs_col, col)
        results_df.loc[results_df.char==col, 'All'] = mi

    # Add ob count by year
    for year in [2018, 2019, 2020, 2021, 2022]:
        results_df.loc[results_df.char=='N', str(year)] = len(df[(df.date.dt.year==year)])

    # Save the table
    with pd.ExcelWriter(out_fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
        results_df.to_excel(writer, sheet_name='raw_char_mi_by_year')

def genCharDescCorrAndRegStats(df: pd.DataFrame, lhs_col: str, out_fp: str) -> None:
    # Build lists of characteristics to produce results for.
    oc_cols = ['char_circulation_tm7',
        'char_circulation_tm30',
        'char_circulation_tm90',
        'char_tx_volume_t',
        'char_tx_volume_tm7',
        'char_addr_new_tm1',
        'char_addr_new_tm7',
        'char_addr_active_tm1',
        'char_addr_active_tm7',
        'char_addr_new_log_delta_tm14_tm7',
        'char_age_destroyed_tm7',
        'char_age_mean_dollar_t',
        'char_delta_flow_dist_tm7',
        'char_delta_holders_dist_tm7',
        'char_prct_supply_in_profit_t']
    ex_cols = ['char_exchange_prct_circ_supply_t',
        'char_cex_prct_circ_supply_t',
        'char_dex_prct_circ_supply_t',
        'char_defi_prct_circ_supply_t',
        'char_traders_prct_circ_supply_t',
        'char_exchange_inflow_tm7',
        'char_exchange_outflow_tm7',
        'char_rank_cmc_t',
        'char_tradable_t']
    social_cols = ['char_social_volume_tm7',
        'char_social_volume_reddit_tm7',
        'char_social_volume_twitter_tm7',
        'char_sent_neg_reddit_tm7',
        'char_sent_neg_twitter_tm7',
        'char_sent_pos_reddit_tm7',
        'char_sent_pos_twitter_tm7',
        'char_sent_volume_consumed_tm7',
        'char_social_dom_avg_tm7',
        'char_dev_activity_tm7',
        'char_vc_t']
    mom_cols = ['char_r_tm7',
        'char_r_tm14',
        'char_r_tm14_tm7',
        'char_r_tm30',
        'char_r_tm60',
        'char_r_tm90',
        'char_r_max_tm7',
        'char_r_max_tm30',
        'char_r_ath_t',
        'char_r_atl_t']
    mic_cols = ['char_trades_t',
        'char_trades_sum_tm7',
        'char_trades_std_tm7',
        'char_volume_sum_tm7',
        'char_volume_std_tm7',
        'char_ask_t',
        'char_bid_t',
        'char_bidask_t',
        'char_ask_size_t',
        'char_bid_size_t',
        'char_illiq_tm7',
        'char_turnover_tm7']
    fin_cols = ['char_price_t',
        'char_size_t',
        'char_mvrv_t',
        'char_alpha_tm7',
        'char_alpha_tm30',
        'char_beta_tm7',
        'char_beta_tm30',
        'char_beta_downside_tm30',
        'char_coskew_tm30',
        'char_iskew_tm30',
        'char_shortfall5_tm7',
        'char_shortfall5_tm90',
        'char_var5_tm7',
        'char_var5_tm90',
        'char_vol_tm7',
        'char_vol_tm30',
        'char_ivol_tm7',
        'char_ivol_tm30']

    # Produce the name of these characteristic groups, in same order
    char_groups = ['onchain', 'exchange', 'social',
        'mom', 'micro', 'financial']
    char_group_cols = [oc_cols, ex_cols, social_cols, 
        mom_cols, mic_cols, fin_cols]
        
    # Output the characteristic descriptive statistics
    genCharDescStatistics(df, char_group_cols, lhs_col, out_fp)

    # Output the correlation table among the pc's
    genPcCorrTable(df, char_group_cols, out_fp)

    # Generate output for each group of characteristics
    for char_name, char_cols in zip(char_groups, char_group_cols):
        # form the relevant data
        char_df = formRelCharDf(df, lhs_col, char_cols, char_name)

        # output the corr table
        genAndOutputCharCorrTable(char_df, char_cols, char_name, out_fp)

        # output the reg table
        genAndOutputCharRegTable(char_df, char_cols, char_name, out_fp)

    # Calculate mutual information measure
    calcMiByYear(df, lhs_col, char_group_cols, out_fp)


In [18]:
def plotMcaps(df: pd.DataFrame, out_fp: str) -> None:
    # Subset to relevant columns
    industry_cols    = [col for col in df.columns if ('industry' in col) & ('industry_t' not in col)]
    asset_usage_cols = [col for col in df.columns if 'asset_usage' in col]
    mcap_df = df[['date', 'asset', 'char_size_t', 'char_pow', 'char_pos']+industry_cols+asset_usage_cols].copy()

    # Create the three dataframes of mcap series to plot
    total_df = mcap_df.groupby('date')[['char_size_t']].sum()
    total_df = total_df.rename(columns={'char_size_t': 'total'})
    asset_df = mcap_df[mcap_df.asset.isin(['btc', 'eth'])].groupby(['date', 'asset'])[['char_size_t']].sum().reset_index()
    asset_df = asset_df.pivot(index='date', columns='asset', values='char_size_t')
    asset_df = total_df.merge(asset_df, on='date', how='inner', validate='one_to_one')

    industry_dfs = []
    for col in industry_cols:
        temp_df = mcap_df[mcap_df[col]==1].groupby('date')[['char_size_t']].sum()
        new_col_name = "_".join(col.split('_')[2:])
        temp_df = temp_df.rename(columns={'char_size_t': new_col_name})
        industry_dfs.append(temp_df)
    industry_df = pd.concat(industry_dfs, axis=1)

    usage_dfs = []
    for col in asset_usage_cols:
        temp_df = mcap_df[mcap_df[col]==1].groupby('date')[['char_size_t']].sum()
        new_col_name = col.split('_')[3]
        temp_df = temp_df.rename(columns={'char_size_t': new_col_name})
        usage_dfs.append(temp_df)
    usage_df = pd.concat(usage_dfs, axis=1)
    usage_df = usage_df.drop('access', axis=1)

    # Create a figure with 3 subplots
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 6))

    # Plot the 'value' column from each dataframe against the 'date' column
    asset_df.plot(ax=axs[0], color=[viridis.colors[0], viridis.colors[255], viridis.colors[177]])
    industry_df.plot(ax=axs[1], color=viridis(np.linspace(0, 1, industry_df.shape[1])))
    usage_df.plot(ax=axs[2], color=viridis(np.linspace(0, 1, usage_df.shape[1])))

    # Set y-scale to logarithmic
    axs[0].set_yscale('log')
    axs[1].set_yscale('log')
    axs[2].set_yscale('log')

    # Remove minor ticks, add vertical grid lines, and remove x-axis labels
    for ax in axs:
        ax.minorticks_off()
        ax.grid(axis='x', which='major')
        ax.tick_params(axis='x', labelbottom=True)
        ax.set_xlabel('')
        ax.set_facecolor('none')

    # Move legends to below the plots
    axs[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, frameon=False)
    axs[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, frameon=False)
    axs[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, frameon=False)

    # Adjust layout
    plt.tight_layout()
    fig.patch.set_visible(False)

    # output
    plt.savefig(out_fp)
    plt.close()


In [ ]:
if __name__ == "__main__":
    # set args
    PANEL_IN_FP      = '../data/clean/panel_weekly.pkl' 
    ASSET_IN_FP      = '../data/clean/asset_universe_dict.pickle'
    MCAP_OUT_FP      = '../output/desc_stats/mcaps.png'
    HIST_OUT_FP      = '../output/desc_stats/histograms.png'
    CUM_RET_OUT_FP   = '../output/desc_stats/cumulative_returns.png'
    SHARPE_OUT_FP    = '../output/desc_stats/sharpe.png'
    TX_OUT_FP        = '../output/desc_stats/btc_tx.png'
    HODL_OUT_FP      = '../output/desc_stats/hodl.png'
    CORR_OUT_FP      = '../output/desc_stats/corr.png'
    RISK_RTRN_OUT_FP = '../output/desc_stats/risk_return.png'
    OUT_FP           = '../output/desc_stats/descriptive_statistics.xlsx'
    PERIODS_IN_YEAR  = 52
    LHS_COL          = 'r_ex_tp7'

    # import
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    df = pd.read_pickle(PANEL_IN_FP)

    # drop rows that are not in the asset universe
    df = subsetToAssetUniverse(df, asset_universe_dict)

    # generate plots
    plotMcaps(df, MCAP_OUT_FP)
    plotReturnHistograms(df, HIST_OUT_FP)
    plotCumulativeReturns(df, CUM_RET_OUT_FP)
    plotRollingSharpe(SHARPE_OUT_FP)
    plotTransactionStats(df, TX_OUT_FP)
    plotHodlingStats(df, HODL_OUT_FP)
    plotRollingCorrelations(CORR_OUT_FP)
    formRiskReturnScatterPlot(df, RISK_RTRN_OUT_FP, PERIODS_IN_YEAR)

    # generate tables
    genSummaryStatistics(df, LHS_COL, OUT_FP, PERIODS_IN_YEAR)
    genForkStatistics(df, OUT_FP) # NOTE: run time is about 5 min.
    genCorrStatistics(df, OUT_FP)
    genCharDescCorrAndRegStats(df, LHS_COL, OUT_FP)

    # report statistics
    reportICOStatistics(df)
    inflationStatistics(df, OUT_FP)
    
    # # TODO SCOPE IF RESULTS FOR ALL CHANGE MUCH AFTER A WINSOR
    # p1 = df[LHS_COL].quantile(0.01)
    # p99 = df[LHS_COL].quantile(0.99)
    # df.loc[df[LHS_COL] < p1, LHS_COL] = p1 
    # df.loc[df[LHS_COL] > p99, LHS_COL] = p1 
